In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import h5py
import pandas as pd
import math
from tqdm import tqdm
from scipy.interpolate import NearestNDInterpolator
import gc
import torch
from torch import nn, einsum
import torch.nn.functional as F
from torch.optim import Adam
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import Compose, ToTensor, Lambda, ToPILImage
from torchvision.utils import save_image
# !pip install einops
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
import psutil

/Users/mt1516/anaconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/mt1516/anaconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <6A7076EE-85BD-37A7-BC35-1D4867F2B3D3> /Users/mt1516/anaconda3/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <F2FE5CF8-5B5B-3FAD-ADF8-C77D90F49FC9> /Users/mt1516/anaconda3/lib/python3.10/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# # %cd "/content/drive/My Drive/Colab Notebooks/MDEuDM/"
# %cd "/content/"

In [3]:
# !wget http://horatio.cs.nyu.edu/mit/silberman/nyu_depth_v2/nyu_depth_v2_labeled.mat -P ./dataset/NYUDv2/
# # sku_mat = h5py.File('./drive/My Drive/Colab Notebooks/MDEuDM/dataset/NYUDv2/nyu_depth_v2_labeled.mat', 'r')
# sku_mat = h5py.File('./dataset/NYUDv2/nyu_depth_v2_labeled.mat', 'r')

### Data Prepressing

Inpaint training data of missing values with nearest neighbors.

In [4]:
# def inpaint_missing_processor(data, rgb=False):
#     """Inpaint missing depth values using nearest neighbor interpolation"""
#     # Create a mask of values
#     if rgb:
#         mask = data == 255
#     else:
#         mask = data == 0
#     # Create a nearest neighbor interpolator
#     interp = NearestNDInterpolator(np.argwhere(~mask), np.array(data)[~mask])
#     # Interpolate the values
#     infilled = interp(np.argwhere(mask))
#     # Replace the zeros with the interpolated values
#     if rgb:
#         data[mask] = infilled.astype(np.uint8)
#     else:
#         data[mask] = infilled.astype(np.float32)
#     return data

# def inpaint_missing(data, rgb=False):
#     if rgb:
#         for i in range(3):
#             data[:,:,i] = inpaint_missing_processor(data[:,:,i], rgb=True)
#         return data
#     return inpaint_missing_processor(data)


# def horizontal_flip(data, rgb=False):
#     if rgb:
#         for i in range(3):
#             data[:,:,i] = np.fliplr(data[:,:,i])
#         return data
#     return np.fliplr(data)
# # # testing = sku_mat['images'][6].T
# # # save the image
# # # import skimage.io as io
# # # io.imsave('./dataset/NYUDv2/image_cleaned/' + str(6) + '.png', testing)
# # # # io.imshow(testing)
# # # # plt.show()
# # # # testing_ = inpaint_missing(testing)
# # # # io.imshow(testing_)
# # # # plt.show()
# # # # io.imshow(sku_mat['depths'][6].T)
# # # # plt.show()
# # # # image = sku_mat['images'][6].T
# # # # io.imshow(image)
# # # # plt.show()
# # # # image_ = inpaint_missing(image, rgb=True)
# # # # io.imshow(image_)
# # # # plt.show()

# for i, depth_photo in enumerate(sku_mat['rawDepths']):
#     depth_photo = inpaint_missing(depth_photo.T)
#     # io.imsave('./dataset/NYUDv2/depth_cleaned/' + str(i) + '.png', depth_photo)
#     depth_photo = cv2.resize(depth_photo, (320, 240), interpolation=cv2.INTER_NEAREST)
#     np.save('./dataset/NYUDv2/depth_cleaned_resized/' + str(i) + '.npy', depth_photo)
#     np.save('./dataset/NYUDv2/depth_cleaned_resized/' + str(i) + '_flipped.npy', horizontal_flip(depth_photo))
#     # np.save('./drive/MyDrive/Colab Notebooks/MDEuDM/dataset/NYUDv2/depth_cleaned_resized/' + str(i) + '.npy', depth_photo)
#     # np.save('./drive/MyDrive/Colab Notebooks/MDEuDM/dataset/NYUDv2/depth_cleaned_resized/' + str(i) + '_flipped.npy', horizontal_flip(depth_photo))

# for i, rgb_photo in enumerate(sku_mat['images']):
#     rgb_photo = inpaint_missing(rgb_photo.T, rgb=True)
#     # io.imsave('./dataset/NYUDv2/image_cleaned' + str(i) + '.png', rgb_photo)
#     rgb_photo = cv2.resize(rgb_photo, (320, 240), interpolation=cv2.INTER_NEAREST)
#     np.save('./dataset/NYUDv2/image_cleaned_resized/' + str(i) + '.npy', rgb_photo)
#     np.save('./dataset/NYUDv2/image_cleaned_resized/' + str(i) + '_flipped.npy', horizontal_flip(rgb_photo, rgb=True))
#     # np.save('./drive/MyDrive/Colab Notebooks/MDEuDM/dataset/NYUDv2/image_cleaned_resized/' + str(i) + '.npy', rgb_photo)
#     # np.save('./drive/MyDrive/Colab Notebooks/MDEuDM/dataset/NYUDv2/image_cleaned_resized/' + str(i) + '_flipped.npy', horizontal_flip(rgb_photo, rgb=True))

### Zip

In [5]:
# !zip -r ./depth_cleaned_resized.zip ./dataset/NYUDv2/depth_cleaned_resized
# !zip -r ./image_cleaned_resized.zip ./dataset/NYUDv2/image_cleaned_resized

In [6]:
# !unzip "./depth_cleaned_resized.zip" -d "./drive/MyDrive/Colab Notebooks/MDEuDM/dataset/NYUDv2/depth_cleaned_resized/"
# !unzip "./image_cleaned_resized.zip" -d "./drive/MyDrive/Colab Notebooks/MDEuDM/dataset/NYUDv2/image_cleaned_resized/"

### Model

In [7]:
class Upsample(nn.Module):
    '''
    Upsampling module
    '''
    # @torch.no_grad()
    def __init__(self, in_channels, out_channels, scale_factor=2, mode='nearest'):
        super().__init__()
        self.scale_factor = scale_factor
        self.mode = mode
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)

    # @torch.no_grad()
    def forward(self, x):
        x = F.interpolate(x, scale_factor=self.scale_factor, mode=self.mode)
        x = self.conv(x)
        return x


class Downsample(nn.Module):
    '''
    Downsampling module
    '''
    # @torch.no_grad()
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)

    # @torch.no_grad()
    def forward(self, x):
        x = self.conv(x)
        return x


class SinusoidalPositionEmbeddings(nn.Module):
    '''
    Time embedding module
    '''
    def __init__(self, dim=1024):
        super(SinusoidalPositionEmbeddings, self).__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = time[:, None] * emb[None, :]
        emb = torch.cat([emb.sin(), emb.cos()], dim=-1)
        return emb


class SelfAttention(nn.Module):
    '''
    Attention module
    '''
    def __init__(self, channel, heads=8):
        super(SelfAttention, self).__init__()
        self.heads = heads
        # hidden_dim = channel * 2
        self.scale = (channel * 2 // heads) ** -0.5
        self.norm = nn.GroupNorm(heads, channel)
        # self.norm = nn.GroupNorm(heads, hidden_dim)
        self.to_qkv = nn.Conv2d(channel, channel * 3, 1, bias=False)
        # self.to_qkv = nn.Conv2d(channel, hidden_dim * 3, 1, bias=False)
        self.to_out = nn.Conv2d(channel * 2, channel, 1)

    def forward(self, x):
        b, c, h, w = x.shape
        x = self.norm(x)
        qkv = self.to_qkv(x).chunk(3, dim=1)
        q, k, v = map(
            lambda t: rearrange(t, 'b (h c) x y -> b h c (x y)', h=self.heads), qkv
        )
        q = q * self.scale
        sim = einsum('b h d i, b h d j -> b h i j', q, k)
        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)
        out = einsum('b h i j, b h d j -> b h i d', attn, v)
        out = rearrange(out, 'b h (x y) d -> b (h d) x y', x=h, y=w)
        return self.to_out(out)


# class LinearAttention(nn.Module):
#     '''
#     Linear attention block
#     '''
#     def __init__(self, dim, heads=8):
#         super(LinearAttention, self).__init__()
#         self.heads = heads
#         hidden_dim = dim * 2
#         self.scale = (hidden_dim // heads) ** -0.5
#         self.to_qkv = nn.Linear(dim, hidden_dim * 3, 1, bias=False)
#         self.to_out = nn.Linear(hidden_dim, dim)

#     def forward(self, x):
#         b, c, h, w = x.shape
#         qkv = self.to_qkv(x).chunk(3, dim=1)
#         q, k, v = map(
#             lambda t: rearrange(t, 'b (h c) x y -> b h c (x y)', h=self.heads), qkv
#         )
#         q = q.softmax(dim=-2)
#         k = k.softmax(dim=-1)
#         q = q * self.scale
#         context = einsum('b h d n, b h e n -> b h d e', k, v)
#         out = einsum('b h d e, b h d n -> b h e n', context, q)
#         out = rearrange(out, 'b h c (x y) -> b (h c) x y', h=self.heads, x=h, y=w)
#         return self.to_out(out)


# class WeightStandardizedConv2d(nn.Conv2d):
#     '''
#     Weight standardized convolutional layer
#     '''
#     def forward(self, x):
#         eps = 1e-5 if x.dtype == torch.float32 else 1e-3
#         weight = self.weight
#         mean = reduce(weight, 'o ... -> o 1 1 1', 'mean')
#         var = reduce(weight, 'o ... -> o 1 1 1', partial(torch.var, unbiased=False))
#         normalized_weight = (weight - mean) / (var + eps).rsqrt()
#         return F.conv2d(x, normalized_weight, self.bias, self.stride, self.padding, self.dilation, self.groups)


class ResNetBlock(nn.Module):
    '''
    Residual convolutional block with GroupNorm and SiLU activation
    '''
    def __init__(self, channels, groups=8):
        super(ResNetBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.GroupNorm(groups, channels),
            nn.SiLU(),
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.GroupNorm(groups, channels),
            nn.SiLU(),
            nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        )
        self.residual = nn.Conv2d(channels, channels, kernel_size=1)

    def forward(self, x):
        return self.conv_block(x) + self.residual(x)


class EncoderBlock(nn.Module):
    '''
    Encoder: Downward convolutional block with GroupNorm and SiLU activation
    '''
    def __init__(self, in_channels, out_channels, strides=(2,2), num_resnet_block=8, allow_conv=False, allow_attention=False):
        super(EncoderBlock, self).__init__()
        self.conv = Upsample(in_channels, out_channels) if allow_conv else None
        self.mlp = nn.Sequential(
            nn.SiLU(),
            nn.Linear(256, out_channels * 2)
        )
        self.scaling = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.GroupNorm(8, out_channels)
        )
        self.resnet_blocks = nn.ModuleList([ResNetBlock(out_channels) for _ in range(num_resnet_block)])
        self.attention = SelfAttention(out_channels) if allow_attention else None
        

    def forward(self, x, time_emb=None):
        if self.conv is not None:
            # print('\t', 'Conv_start', psutil.virtual_memory()[1]/1024000000)
            x = self.conv(x)
            # print('\t', 'Conv_done', psutil.virtual_memory()[1]/1024000000)
        if self.mlp is not None:
        # print('\t', 'mlp_start', psutil.virtual_memory()[1]/1024000000)
            time_emb = self.mlp(time_emb)
            time_emb = rearrange(time_emb, 'b c -> b c 1 1')
            scale_shift = time_emb.chunk(2, dim=1)
            x = self.scaling(x) * (scale_shift[0] + 1) + scale_shift[1]
            x = nn.SiLU()(x)
        # print('\t', 'mlp_done', psutil.virtual_memory()[1]/1024000000)
        for block in self.resnet_blocks:
            # print('\t', 'block_start', psutil.virtual_memory()[1]/1024000000)
            x = block(x)
            # print('\t', 'block_done', psutil.virtual_memory()[1]/1024000000)
        if self.attention is not None:
            # print('\t', 'attention_start', psutil.virtual_memory()[1]/1024000000)
            x = self.attention(x)
            # print('\t', 'attention_done', psutil.virtual_memory()[1]/1024000000)
        return x


class DecoderBlock(nn.Module):
    '''
    Decoder: Upward convolutional block with GroupNorm and SiLU activation
    '''
    def __init__(self, in_channels, out_channels, strides=(2,2), num_resnet_block=8, allow_conv=False, allow_attention=False):
        super(DecoderBlock, self).__init__()
        self.mlp = nn.Sequential(
            nn.SiLU(),
            nn.Linear(256, in_channels * 2)
        )
        self.scaling = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1),
            nn.GroupNorm(8, in_channels)
        )
        self.resnet_blocks = nn.ModuleList([ResNetBlock(in_channels) for _ in range(num_resnet_block)])
        self.attention = SelfAttention(in_channels) if allow_attention else None
        self.conv = Downsample(in_channels, out_channels) if allow_conv else None
        # self.in_channels = in_channels

    def forward(self, x, time_emb=None):
        if self.mlp is not None:
        # print('\t', 'mlp_start', psutil.virtual_memory()[1]/1024000000)
            time_emb = self.mlp(time_emb)
            time_emb = rearrange(time_emb, 'b c -> b c 1 1')
            scale_shift = time_emb.chunk(2, dim=1)
            x = self.scaling(x) * (scale_shift[0] + 1) + scale_shift[1]
            x = nn.SiLU()(x)
        # time_emb.detach()
        # print('\t', 'mlp_done', psutil.virtual_memory()[1]/1024000000)
        for block in self.resnet_blocks:
            # print('\t', 'block_start', psutil.virtual_memory()[1]/1024000000)
            x = block(x)
            # print('\t', 'block_done', psutil.virtual_memory()[1]/1024000000)
        if self.attention is not None:
            # print('\t', 'attention_start', psutil.virtual_memory()[1]/1024000000)
            x = self.attention(x)
            # print('\t', 'attention_done', psutil.virtual_memory()[1]/1024000000)
        if self.conv is not None:
            # print('\t', 'Conv_start', psutil.virtual_memory()[1]/1024000000)
            # print(x.shape)
            x = self.conv(x)
            # print(x.shape)
            # print('\t', 'Conv_done', psutil.virtual_memory()[1]/1024000000)
        return x


class EfficientUNet(nn.Module):
    '''
    Efficient UNet architecture
    '''
    def __init__(
            self,
            in_channels=4,
            out_channels=1,
            channels=[64,128,256,512,1024],
            num_resnet_blocks=[1,2,4,8,8],
            skips=[True,True,True,True,False],
            # channels=[128,256],
            # num_resnet_blocks=[1,1],
            # skips=[True,False]
        ):
        super(EfficientUNet, self).__init__()
        self.skips = skips
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()

        out_chan = 32
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(out_chan),
            nn.Linear(out_chan, 256),
            nn.GELU(),
            nn.Linear(256, 256),
        )

        # Input - Initial convolution
        self.input = nn.Conv2d(in_channels, out_chan, kernel_size=3, padding=1)

        # Down part
        in_chan = out_chan
        for num_resnet_block, channel, skip in zip(num_resnet_blocks[:-1], channels[:-1], skips[:-1]):
            out_chan = channel
            self.downs.append(EncoderBlock(in_chan, out_chan, num_resnet_block=num_resnet_block, allow_conv=True))
            # self.downs.append(EncoderBlock(in_chan, out_chan, num_resnet_block=num_resnet_block, allow_conv=True, allow_attention=not skip))
            in_chan = out_chan

        # Middle part
        out_chan = channels[-1]
        self.middle = EncoderBlock(in_chan, out_chan, num_resnet_block=8, allow_conv=True, allow_attention=True)
        in_chan = out_chan

        # Up part
        for num_resnet_block, channel, skip in zip(reversed(num_resnet_blocks[:-1]), reversed(channels[:-1]), reversed(skips[:-1])):
            out_chan = channel
            self.ups.append(DecoderBlock(in_chan, out_chan, num_resnet_block=num_resnet_block, allow_conv=True))
            in_chan = out_chan

        # Output - Compress to 1 channel
        self.output = nn.Linear(in_chan, out_channels)

    def forward(self, x, time):
        # print('UNet Start', psutil.virtual_memory()[1]/1024000000)
        skip_connections = []
        t = self.time_mlp(time)
        # print('Time MLP', psutil.virtual_memory()[1]/1024000000)
        x = self.input(x)
        for skip, down in zip(self.skips, self.downs):
            # print('Down', psutil.virtual_memory()[1]/1024000000)
            x = down(x, t)
            if skip:
                skip_connections.append(x)
            else:
                skip_connections.append(None)
        # print(skip_connections[0].shape)
        for skip, up in zip(reversed(skip_connections), self.ups):
            # print('Up', psutil.virtual_memory()[1]/1024000000)
            if skip is not None:
                x = torch.cat((x, skip), dim=1)
            x = up(x, t)
        # print(x.shape)
        x = self.output(x.transpose(1, 2).transpose(2, 3)).transpose(2,3).transpose(1,2)
        # print(x.shape)
        # print('UNet End', psutil.virtual_memory()[1]/1024000000)

        return x

In [9]:
def extract(a, t, x_shape):
    batch_size = t.shape[0]
    t = t.to(a.device)
    out = a.gather(-1, t)
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1)))

# def noise_like(shape, device, repeat=False):
#     def repeat_noise(): return torch.randn(
#         (1, *shape[1:]), device=device).repeat(shape[0], *((1,) * (len(shape) - 1)))

#     def noise(): return torch.randn(shape, device=device)

#     return repeat_noise() if repeat else noise()

def warm_up_beta(linear_start, linear_end, timesteps, warm_up_frac):
    betas = linear_end * torch.ones(timesteps, dtype=np.float32)
    warm_up_steps = int(timesteps * warm_up_frac)
    betas[:warm_up_steps] = torch.linspace(linear_start, linear_end, warm_up_steps, dtype=np.float32)
    return betas

def make_beta_schedule(schedule, timesteps, linear_start=1e-4, linear_end=2e-2, cosine_s=8e-3):
    if schedule == 'warmup10':
        betas = warm_up_beta(linear_start, linear_end, timesteps, 10)
    elif schedule == 'warmup20':
        betas = warm_up_beta(linear_start, linear_end, timesteps, 20)
    elif schedule == 'warmup50':
        betas = warm_up_beta(linear_start, linear_end, timesteps, 50)
    elif schedule == 'linear':
        betas = torch.linspace(linear_start, linear_end, timesteps, dtype=np.float32)
    elif schedule == 'quad':
        betas = torch.linspace(linear_start ** 0.5, linear_end ** 0.5, timesteps, dtype=np.float32) ** 2
    elif schedule == 'const':
        betas = linear_end * torch.ones(timesteps, dtype=np.float32)
    elif schedule == 'cosine':
        steps = timesteps + 1
        x = torch.linspace(0, timesteps, steps)
        alphas = torch.cos(((x / timesteps) + cosine_s) / (1 + cosine_s) * torch.pi * 0.5) ** 2
        alphas = alphas / alphas[0]
        betas = 1 - (alphas[1:] / alphas[:-1])
        betas = torch.clip(betas, linear_start, 0.999)
    else:
        raise ValueError('Unknown beta schedule')

    return betas


class StepUnrolledDiffusion(nn.Module):
    def __init__(self, schedule_name, model, optimizer, loss_fn, device, timesteps=1000):
        super(StepUnrolledDiffusion, self).__init__()
        self.schedule = make_beta_schedule(schedule_name, timesteps)
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.device = device
        self.timesteps = timesteps

        betas = self.schedule.detach().cpu().numpy() if isinstance(self.schedule, torch.Tensor) else self.schedule
        alphas = 1. - betas
        alphas_cumprod = np.cumprod(alphas, 0)
        alphas_cumprod_prev = np.append(1., alphas_cumprod[:-1])
        posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)

        self.betas = torch.from_numpy(betas).to(device)
        self.alphas_cumprod = torch.from_numpy(alphas_cumprod).to(device)
        # self.alphas_cumprod_prev = torch.from_numpy(alphas_cumprod_prev).to(device)
        self.sqrt_alphas_cumprod = torch.sqrt(self.alphas_cumprod)
        self.sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - self.alphas_cumprod)
        # self.log_one_minus_alphas_cumprod = torch.log(1. - self.alphas_cumprod)
        self.sqrt_recip_alphas_cumprod = torch.from_numpy(np.sqrt(1. / alphas_cumprod)).to(device)
        self.sqrt_recipm1_alphas_cumprod = torch.sqrt(1. / self.alphas_cumprod - 1)
        self.posterior_variance = torch.from_numpy(posterior_variance).to(device)
        self.posterior_log_variance_clipped = torch.log(torch.clamp(self.posterior_variance, min=1e-20))
        self.posterior_mean_coef1 = torch.from_numpy(betas * np.sqrt(alphas_cumprod_prev) / (1. - alphas_cumprod)).to(device)
        self.posterior_mean_coef2 = torch.from_numpy((1. - alphas_cumprod_prev) * np.sqrt(alphas) / (1. - alphas_cumprod)).to(device)

    @torch.no_grad()
    def denoise(self, x, t):
        mean = extract(self.sqrt_alphas_cumprod, t, x.shape) * x
        variance = 1. - self.alphas_cumprod
        log_variance = self.posterior_log_variance_clipped
        noise = torch.randn(x.shape, device=self.device, dtype=torch.float32)
        x = (mean + self.sqrt_recipm1_alphas_cumprod * noise * torch.exp(0.5 * log_variance)).to(torch.float32)
        for i in reversed(range(self.timesteps)):
            output = self.model(x, torch.full((x.shape[0],), i, device=self.device, dtype=torch.long))
            x = (x - self.posterior_mean_coef1[i] * output - self.posterior_mean_coef2[i] * output) / self.sqrt_one_minus_alphas_cumprod[i]
        return x

    # def denoise_depth(self, x):
    #     rgb_channels, depth_channel = x[:,:3,:,:], x[:,3,:,:]
    #     mean = self.sqrt_recip_alphas_cumprod * depth_channel
    #     variance = 1. - self.alphas_cumprod
    #     log_variance = self.posterior_log_variance_clipped
    #     noise = noise_like(depth_channel.shape, self.device)
    #     depth_channel = mean + self.sqrt_recipm1_alphas_cumprod * noise * torch.exp(0.5 * log_variance)
    #     for i in reversed(range(self.timesteps)):
    #         depth_channel = (depth_channel - self.posterior_mean_coef1[i] * self.model(depth_channel, i) - self.posterior_mean_coef2[i] * self.model(depth_channel, i)) / self.sqrt_one_minus_alphas_cumprod[i]
    #     return torch.cat((rgb_channels, depth_channel), dim=1)


    # def q_mean_variance(self, t, x_start):
    #     # rgb_channels, depth_channel = x_start[:,:3,:,:], x_start[:,3,:,:]
    #     # mean = extract(self.sqrt_alphas_cumprod, t, x_start.shape) * depth_channel
    #     # variance = extract(1. - self.alphas_cumprod, t, x_start.shape)
    #     # log_variance = extract(self.log_one_minus_alphas_cumprod, t, x_start.shape)
    #     # return torch.cat((rgb_channels, mean), dim=1), torch.cat((rgb_channels, variance), dim=1), torch.cat((rgb_channels, log_variance), dim=1)
    #     mean = extract(self.sqrt_alphas_cumprod, t, x_start.shape) * x_start
    #     variance = extract(1. - self.alphas_cumprod, t, x_start.shape)
    #     log_variance = extract(self.log_one_minus_alphas_cumprod, t, x_start.shape)
    #     return mean, variance, log_variance

    # def q_sample(self, x_t, t, noise=None):
    #     if noise is None:
    #         noise = torch.randn_like(x_t).to(self.device).to(torch.float32)
    #     return extract(self.sqrt_alphas_cumprod, t, x_t.shape) * x_t + extract(self.sqrt_one_minus_alphas_cumprod, t, x_t.shape) * noise

    @torch.no_grad()
    def q_posterior(self, x_start, x_t, t):
        coef1 = extract(self.posterior_mean_coef1, t, x_t.shape)
        posterior_mean = (extract(self.posterior_mean_coef1, t, x_t.shape) * x_start + extract(self.posterior_mean_coef2, t, x_t.shape) * x_t)
        posterior_variance = extract(self.posterior_variance, t, x_t.shape)
        posterior_log_variance_clipped = extract(self.posterior_log_variance_clipped, t, x_t.shape)
        return posterior_mean, posterior_variance, posterior_log_variance_clipped

    @torch.no_grad()
    def p_mean_variance(self, x, t, clip_denoised=False):
        x_recon = extract(self.sqrt_recip_alphas_cumprod, t, x.shape) * x - extract(self.sqrt_recipm1_alphas_cumprod, t, x.shape) * self.denoise(x, t)
        if clip_denoised:
            x_recon = torch.clamp(x_recon, -1., 1.)
        model_mean, model_variance, model_log_variance = self.q_posterior(x, x_recon, t)
        return model_mean, model_variance, model_log_variance

    @torch.no_grad()
    def p_sample(self, x, t, clip_denoised=False, repeat_noise=False):
        # rgb_channels, depth_channel = x[:,:3,:,:], x[:,3,:,:]
        # batch, *_, device = *depth_channel.shape, depth_channel.device
        # mean, _, log_variance = self.p_mean_variance(depth_channel, t, clip_denoised)
        # noise = noise_like(depth_channel.shape, device, repeat=repeat_noise)
        # nonzero_mask = (1. - (t == 0).float()).reshape(batch, *((1,) * len(depth_channel.shape - 1)))
        # if t == torch.tensor([0]):
        #     return torch.cat((rgb_channels, mean), dim=1)
        # return torch.cat((rgb_channels, mean + torch.exp(0.5 * log_variance) * noise * nonzero_mask), dim=1)
        # batch, device = x.shape[0], x.device
        model_mean, _, model_log_variance = self.p_mean_variance(x, t, clip_denoised=clip_denoised)
        noise = torch.randn((1, *x.shape[1:]), device=x.device).repeat(x.shape[0], *((1,) * (len(x.shape) - 1)))
        nonzero_mask = (1. - (t == 0).float()).reshape(x.shape[0], *((1,) * (len(x.shape) - 1)))
        # if t == torch.tensor([0]):
        #     return model_mean
        return model_mean + torch.exp(0.5 * model_log_variance) * noise * nonzero_mask

    @torch.no_grad()
    def p_sample_loop(self, x, denoising_steps, sample_interval):
        device = self.betas.device
        img = torch.randn(x.shape, device=device)
        ret_img = img[:,3,:,:].unsqueeze(1)
        # for i in tqdm(reversed(range(denoising_steps)), desc='sampling loop timestep', total=denoising_steps):
        for i in reversed(range(denoising_steps)):
            img = self.p_sample(img, torch.full((x.shape[0],), i, device=device, dtype=torch.long), clip_denoised=True)[:,3,:,:].unsqueeze(1)
            if (i+1) % sample_interval == 0:
                ret_img = torch.cat([ret_img, img], dim=0)
        return ret_img

    @torch.no_grad()
    def sample(self, rgb_photo, denoising_steps, sample_interval):
        return self.p_sample_loop(torch.cat((rgb_photo.to(self.betas.device), torch.randn((rgb_photo.shape[0], 1, rgb_photo.shape[2], rgb_photo.shape[3]), device=self.betas.device)), dim=1), denoising_steps, sample_interval)

    def forward(self, x, timesteps=1000, finetune=False):
        rgb_channels, depth_channel = x[:,:3,:,:], x[:,3,:,:].unsqueeze(1)
        if not finetune:
            for i in range(timesteps):
                # print('forward', i, psutil.virtual_memory()[1]/1024000000)
                noise = torch.randn(depth_channel.shape, device=self.device)
                depth_channel = depth_channel + torch.sqrt(self.betas[i]).to(self.device) * noise
                # print('Pre-pred', psutil.virtual_memory()[1]/1024000000)
                pred_depth = self.model(torch.cat((rgb_channels, depth_channel), dim=1), torch.full((x.shape[0],), i, device=self.device, dtype=torch.long))
            return pred_depth
        else:
            pred_depth = self.model(x, torch.full((x.shape[0],), 0, device=self.device, dtype=torch.long))
            for i in range(1, timesteps):
                noise = torch.randn(depth_channel.shape, device=self.device)
                pred_depth = pred_depth + torch.sqrt(self.betas[i]) * torch.randn(depth_channel.shape, device=self.device)
                pred_depth = self.model(torch.cat((rgb_channels, pred_depth), dim=1), torch.full((x.shape[0],), i, device=self.device, dtype=torch.long))
            return pred_depth

    # def step(self, x, y, finetune=False):
    #     loss = self.loss_fn(self.forward(x, finetune=finetune), y)
    #     loss.backward()
    #     self.optimizer.step()
    #     return loss

    def train(self, train_loader, epochs=10, timesteps=1000, finetune=False):
        self.model.train()
        # print('Train begin', psutil.virtual_memory()[1]/1024000000)
        for epoch in range(epochs):
            gc.collect()
            torch.cuda.empty_cache()
            for i, batch in tqdm(enumerate(train_loader), desc=f'Training epoch {epoch+1}', total=len(train_loader), unit='batch'):
            # for i, batch in enumerate(train_loader):
                self.optimizer.zero_grad()
                image = batch[0].to(self.device)
                depth = batch[1].to(self.device)
                image_with_depth = Variable(torch.cat((image, depth), dim=1), requires_grad=True)
                depth = Variable(depth, requires_grad=True)
                output = self.forward(image_with_depth, timesteps=timesteps, finetune=finetune)
                loss = self.loss_fn(output, depth)
                loss.backward()
                self.optimizer.step()
                print(f'Epoch: {epoch+1}, Batch: {i}, Loss: {loss.item()}')
            if not (epoch + 1) % 10:
                torch.save(self.model.state_dict(), f'./models/epoch_{epoch+1}.pth')

    @torch.no_grad()
    def test(self, test_loader, denoising_steps=128, sample_interval=32):
        self.model.eval()
        pred_list = []
        depth_list = []
        # for i, batch in tqdm(enumerate(test_loader), desc='Testing', total=len(test_loader), unit='batch'):
        for i, batch in enumerate(test_loader):
            image = batch[0].to(self.device)
            depth = batch[1].to(self.device)
            pred = self.sample(image, denoising_steps=denoising_steps, sample_interval=sample_interval)[::2,:,:,:]
            np.save(f'./results/pred_{i}.npy', pred.cpu().numpy())
            pred_list.append(pred)
            np.save(f'./results/depth_{i}.npy', depth.cpu().numpy())
            depth_list.append(depth)
            # for j in range(batch_size):
            #     np.save(f'./results/pred_{i*batch_size+j}.npy', pred[j].cpu().numpy())
            #     np.save(f'./results/depth_{i*batch_size+j}.npy', depth[j].cpu().numpy())
                # pred_list.append(pred[j].cpu().numpy())
                # depth_list.append(depth[j].cpu().numpy())
            
            # save the predicted depth map
            # pred = pred.cpu().numpy()
            # depth = depth.cpu().numpy()
            # for j in range(pred.shape[0]):
            #     np.save(f'./results/pred_{i*8+j}.npy', pred[j])
            #     np.save(f'./results/depth_{i*8+j}.npy', depth[j])


        pred = np.stack([pred_item.squeeze(1).cpu().numpy() for pred_item in pred_list])
        depth = np.stack([depth_item.squeeze(1).cpu().numpy() for depth_item in depth_list])

        # accuracy under the threshold 1.25^i, i = 1, 2, 3, min=0, max=1
        print('Accuracy under threshold δ_i < 1.25^i')
        for i in range(1, 4):
            threshold = 1.25 ** i
            accuracy = np.sum(np.maximum(pred / depth, depth / pred) < threshold) / pred.shape[0] / pred.shape[1] / pred.shape[2]
            print(f'δ_{i}: {accuracy} (higher is better)')
        
        # absolute relative error
        rel = np.sum(np.abs(pred - depth) / depth) / pred.shape[0] / pred.shape[1] / pred.shape[2]
        print(f'Absolute relative error: {rel} (lower is better)')

        # root mean squared error
        rms = np.sqrt(np.sum((pred - depth) ** 2) / pred.shape[0] / pred.shape[1] / pred.shape[2])
        print(f'Root mean squared error: {rms} (lower is better)')

        # # absolute error of log depth
        # abs_log = np.sum(np.log(np.abs(pred / depth))) / pred.shape[0] / pred.shape[1] / pred.shape[2]
        # print(f'Absolute error of log depth: {abs_log} (lower is better)')
            

class NYUDv2Dataset(Dataset):
    def __init__(self, device='cpu', sample=[0,2899]):
        super(NYUDv2Dataset, self).__init__()
        images = {'image': [], 'depth': []}
        image_dir = './dataset/NYUDv2/image_cleaned_resized/'
        depth_dir = './dataset/NYUDv2/depth_cleaned_resized/'
        # image_dir = './drive/MyDrive/Colab Notebooks/MDEuDM/dataset/NYUDv2/image_cleaned_resized/'
        # depth_dir = './drive/MyDrive/Colab Notebooks/MDEuDM/dataset/NYUDv2/depth_cleaned_resized/'
        for file in os.listdir(image_dir)[sample[0]:sample[1]]:
            if file.endswith('.npy'):
                image = np.load(image_dir + file).transpose(2, 0, 1)
                image = torch.from_numpy(image)
                images['image'].append(image)
        for file in os.listdir(depth_dir)[sample[0]:sample[1]]:
            if file.endswith('.npy'):
                depth = np.load(depth_dir + file)
                depth = torch.from_numpy(depth).unsqueeze(0)
                images['depth'].append(depth)
        self.x = torch.stack(images['image']).to(device)
        self.y = torch.stack(images['depth']).to(device)
        self.n_samples = len(self.x)

    def __getitem__(self, index):
        return [self.x[index], self.y[index]]

    def __len__(self):
        return self.n_samples

In [10]:
gc.enable()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# training = NYUDv2Dataset(sample=[0, 2], device=device)
# training_loader = DataLoader(training, batch_size=2, shuffle=False)
# finetuning = NYUDv2Dataset(sample=[2, 4], device=device)
# finetuning_loader = DataLoader(finetuning, batch_size=2, shuffle=False)
# testing = NYUDv2Dataset(sample=[4, 6], device=device)
# testing_loader = DataLoader(testing, batch_size=2, shuffle=False)
training = NYUDv2Dataset(sample=[0, 2720], device=device)
training_loader = DataLoader(training, batch_size=8, shuffle=False)
# finetuning = NYUDv2Dataset(sample=[2720, 2800], device=device)
# finetuning_loader = DataLoader(finetuning, batch_size=8, shuffle=False)
# testing = NYUDv2Dataset(sample=[2850, 2899], device=device)
# testing_loader = DataLoader(testing, batch_size=8, shuffle=False)
timesteps = 1000
unet_model = EfficientUNet()
optimizer = Adam(unet_model.parameters(), lr=1e-4)
loss_fn = nn.SmoothL1Loss()
model = StepUnrolledDiffusion('cosine', unet_model, optimizer, loss_fn, device, timesteps=timesteps)
if torch.cuda.is_available():
    unet_model = nn.DataParallel(unet_model)
    model = nn.DataParallel(model)
print("training start")
# # print(psutil.virtual_memory()[1]/1024000000)
model.train(training_loader, epochs=100, timesteps=timesteps, finetune=False)
# print("finetuning start")
# model.train(finetuning_loader, epochs=1, timesteps=timesteps, finetune=True)
# print("testing start")
# model.test(testing_loader, denoising_steps=1, sample_interval=1)
print("done")

training start
Epoch: 0


: 

: 

In [ ]:
# from PIL import Image

# transform = Compose([
#     ToTensor(),
#     Lambda(lambda t: (t * 2) - 1),
# ])

# reverse_transform = Compose([
#     Lambda(lambda t: (t + 1) / 2),
#     Lambda(lambda t: t.permute(1,2,0)),
#     Lambda(lambda t: t * 255.),
#     Lambda(lambda t: t.numpy().astype(np.uint8)),
#     # ToPILImage(),
# ])

# image = np.load('./drive/MyDrive/Colab Notebooks/MDEuDM/dataset/NYUDv2/depth_cleaned_resized/0.npy')
# image = Image.fromarray(image.astype('uint8')).convert('L')
# # display(image)
# image = transform(image).unsqueeze(0).to(device)

# def get_noisy_image(x, t):
#   x = model.q_sample(x, t.to(device))
#   x = reverse_transform(x.squeeze().cpu())
#   return x

# def plot(imgs, with_orig=False, row_title=None, **imshow_kwargs):
#     if not isinstance(imgs[0], list):
#         # Make a 2d grid even if there's just 1 row
#         imgs = [imgs]

#     num_rows = len(imgs)
#     num_cols = len(imgs[0]) + with_orig
#     fig, axs = plt.subplots(figsize=(320,240), nrows=num_rows, ncols=num_cols, squeeze=False)
#     for row_idx, row in enumerate(imgs):
#         row = [image] + row if with_orig else row
#         for col_idx, img in enumerate(row):
#             ax = axs[row_idx, col_idx]
#             ax.imshow(np.asarray(img), **imshow_kwargs)
#             ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

#     if with_orig:
#         axs[0, 0].set(title='Original image')
#         axs[0, 0].title.set_size(8)
#     if row_title is not None:
#         for row_idx in range(num_rows):
#             axs[row_idx, 0].set(ylabel=row_title[row_idx])

#     plt.tight_layout()

# # plot([get_noisy_image(image, torch.tensor([t])) for t in [0, 99, 199, 299, 399, 499, 599, 699, 799, 899, 999]])
# plot([get_noisy_image(image, torch.tensor([t])) for t in [0, 199, 399, 599, 799, 999]])
# # plot([get_noisy_image(image, torch.tensor([t])) for t in [0, 49, 99, 149, 199]])

RuntimeError: ignored